# ASSIGNMENT 2

## Chin Wai Yee - Data Preprocessing

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tristanzhang32/ai-generated-images-vs-real-images")

ai_dir = path + "/train/fake"
real_dir = path + "/train/real"

# Label 0 = AI-generated, Label 1 = Real
ai_data = [(os.path.join(ai_dir, f), 0) for f in os.listdir(ai_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
real_data = [(os.path.join(real_dir, f), 1) for f in os.listdir(real_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

full_data = ai_data + real_data
labels = [label for _, label in full_data]

In [ ]:
# Train/val/test split
train_val_data, test_data = train_test_split(full_data, test_size=0.15, stratify=labels, random_state=42)
train_data, val_data = train_test_split(train_val_data, test_size=0.15, stratify=[label for _, label in train_val_data], random_state=42)

# Custom Dataset
class ImagePathDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        try:
            image = Image.open(img_path)
    
            if image.mode == 'P':
                image = image.convert('RGBA')
                background = Image.new("RGB", image.size, (255, 255, 255))
                image = Image.alpha_composite(background.convert('RGBA'), image).convert('RGB')
            else:
                image = image.convert('RGB')
    
            if self.transform:
                image = self.transform(image)
            return image, label
    
        except (OSError, IOError) as e:
            print(f"Skipping corrupted image: {img_path} ({str(e)})")
            return self.__getitem__((idx + 1) % len(self.data))  # Try next image

In [ ]:
# Data Augmentation for training
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Standard transform for val/test
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

In [ ]:
# Create datasets and loaders
train_dataset = ImagePathDataset(train_data, transform=train_transform)
val_dataset = ImagePathDataset(val_data, transform=test_transform)
test_dataset = ImagePathDataset(test_data, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

## Loh Kin Ming - Model, Hyperparameters & Training

## Brandon Ting En Junn - Evaluation & Demo